# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
#import CSV file previously used
weather_data = "weather_data_df.csv"
weather_data_df = pd.read_csv(weather_data)
weather_data_df.head()

,Unnamed: 0,cities,lat,long,max_fahrenheit,humidity,cloudiness,wind_speed,country,date
0,0,Cabedelo,-6.98,-34.83,75.20,100,75,8.05,BR,1601259379
1,1,Lata,40.16,-8.33,64.00,87,0,2.17,PT,1601259380
2,2,Harlow,51.78,0.11,52.00,87,90,9.17,GB,1601259380
3,3,Hobart,-42.88,147.33,57.99,50,75,12.75,AU,1601259369
4,4,Belaya Gora,68.53,146.42,46.40,56,100,3.13,RU,1601259381


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
city_coord = weather_data_df[["lat","long"]]

In [4]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout = figure_layout)
markers = gmaps.marker_layer(city_coord)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition.
weather_data_df = weather_data_df.loc[weather_data_df['max_fahrenheit'] >70]
weather_data_df = weather_data_df.loc[weather_data_df['max_fahrenheit'] <80]

weather_data_df = weather_data_df.loc[weather_data_df['wind_speed'] <10]
weather_data_df = weather_data_df.loc[weather_data_df['cloudiness'] ==0]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = weather_data_df.dropna()

hotel_coord = zip(hotel_df["lat"],hotel_df["long"])
radius = 5000
type_ = 'lodging'

hotels = []

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


#for hotel_coord in hotel_coord:
for hotel_coord in hotel_coord:
    try:
        params = {
            'location': (f'{hotel_coord[0]},{hotel_coord[1]}'),
            'radius': radius,
            'type': type_,
            'key':g_key
        }
        hotel_json = requests.get(url, params=params).json()
        hotels.append(hotel_json['results'][0]['name'])
    
    except IndexError:
        print(f"Error with hotel coordinates not found {hotel_coord[0]},{hotel_coord[1]}")
hotels    


error with hotel coordinates 42.6,-73.97


['OlusHome',
 'Florence Hotel Assiut',
 'Pousada dos Navegantes',
 'Hotel Fairmont Makkah Clock Royal Tower',
 'Blackwater Motor Inn',
 'Katsuura Hotel Mikazuki',
 'Casa do Ian😎',
 'New Aegli Resort Hotel 4 stars',
 'Silo Motor Inn',
 'Apart Hotel Rio Xingu',
 'Eureka Inn Trademark By Wyndham',
 'Microtel Inn & Suites by Wyndham Saraland/North Mobile']

In [23]:
drop_missing_hotel = hotel_df.drop([31])
drop_missing_hotel["Hotel Name"] = hotels
drop_missing_hotel

,Unnamed: 0,cities,lat,long,max_fahrenheit,humidity,cloudiness,wind_speed,country,date,Hotel Name
17,17,Kiryat Gat,31.61,34.76,75.99,81,0,4.47,IL,1601259388,OlusHome
28,28,Assiut,27.18,31.18,75.20,44,0,9.17,EG,1601259394,Florence Hotel Assiut
150,150,Caravelas,-17.71,-39.25,71.87,88,0,7.36,BR,1601259468,Pousada dos Navegantes
194,194,Makkah al Mukarramah,21.43,39.83,79.90,46,0,2.13,SA,1601259447,Hotel Fairmont Makkah Clock Royal Tower
283,283,Blackwater,-23.58,148.88,76.50,37,0,9.28,AU,1601259536,Blackwater Motor Inn
336,336,Katsuura,35.13,140.30,78.01,50,0,4.00,JP,1601259566,Katsuura Hotel Mikazuki
356,356,Jacareacanga,-6.22,-57.75,75.96,73,0,0.31,BR,1601259576,Casa do Ian😎
418,418,Poros,37.50,23.45,73.99,74,0,5.01,GR,1601259611,New Aegli Resort Hotel 4 stars
429,429,Biloela,-24.42,150.50,77.41,27,0,0.78,AU,1601259616,Silo Motor Inn
456,456,São Félix do Xingu,-6.64,-51.99,75.43,67,0,2.04,BR,1601259631,Apart Hotel Rio Xingu


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
